# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-3.66,83,75,5.66,CA,1712010833
1,1,tura,25.5198,90.2201,20.29,81,4,2.54,IN,1712010853
2,2,cidreira,-30.1811,-50.2056,23.32,89,4,5.60,BR,1712010853
3,3,adamstown,-25.0660,-130.1015,23.87,77,94,7.74,PN,1712010854
4,4,vila velha,-20.3297,-40.2925,26.97,81,75,3.09,BR,1712010447


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#https://hvplot.holoviz.org/user_guide/Geographic_Data.html
#https://stackoverflow.com/questions/70994916/hvplot-points-with-geo-true-does-not-show-the-map
# %%capture --no-display

# Configure the map plot


mappy=city_data_df.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=.65,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                hover_cols=['Country','Cloudiness','Wind Speed'])

# Display the map
mappy


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#criteria:temperature between 21 to 30,Humidity is less than 80%, windspeed is less then 4m/s, cloudiness<75

crit_df=city_data_df.loc[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']<30) & 
                         (city_data_df['Humidity']<80) & (city_data_df['Wind Speed']<4)
                        &(city_data_df['Cloudiness']<75)].dropna()


# Display sample data
crit_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,trinidad,-14.8333,-64.9000,28.99,79,40,2.57,BO,1712010861
42,42,port hedland,-20.3167,118.5667,27.03,47,1,2.57,AU,1712010863
63,63,san ignacio rio muerto,27.4167,-110.2333,24.29,25,1,3.85,MX,1712010869
76,76,bandarbeyla,9.4942,50.8122,27.58,76,19,3.63,SO,1712010872
100,100,tamanrasset,22.7850,5.5228,22.95,10,0,2.57,DZ,1712010878
107,107,kone,-21.0595,164.8658,28.05,57,15,2.01,NC,1712010880
142,142,portland,45.5234,-122.6762,21.58,43,40,0.89,US,1712010352
146,146,laguna,38.4210,-121.4238,22.99,35,0,3.60,US,1712010305
157,157,hamilton,39.1834,-84.5333,21.35,79,40,2.57,US,1712010427
158,158,marka,1.7159,44.7717,28.26,75,28,3.42,SO,1712010892


#### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=crit_df.loc[:,['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''



# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
34,trinidad,BO,-14.8333,-64.9000,79,
42,port hedland,AU,-20.3167,118.5667,47,
63,san ignacio rio muerto,MX,27.4167,-110.2333,25,
76,bandarbeyla,SO,9.4942,50.8122,76,
100,tamanrasset,DZ,22.7850,5.5228,10,
107,kone,NC,-21.0595,164.8658,57,
142,portland,US,45.5234,-122.6762,43,
146,laguna,US,38.4210,-121.4238,35,
157,hamilton,US,39.1834,-84.5333,79,
158,marka,SO,1.7159,44.7717,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories='accommodation.hotel'
limit=1

params = {'categories':categories,
          'apiKey':geoapify_key,'limit':limit}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
trinidad - nearest hotel: Hotel Palace
port hedland - nearest hotel: The Esplanade Hotel
san ignacio rio muerto - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
kone - nearest hotel: Pacifik Appartels
portland - nearest hotel: the Hoxton
laguna - nearest hotel: Holiday Inn Express & Suites
hamilton - nearest hotel: North Vista Manor
marka - nearest hotel: Salsabiil Hotel
hillsboro - nearest hotel: Residence Inn Portland Hillsboro/Brookwood
ha tinh - nearest hotel: Hong Chau Nha Nhgi
umluj - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
siwa oasis - nearest hotel: فندق الكيلانى
al burayqah - nearest hotel: فندق ماس ليبيا
arraial do cabo - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
kavaratti - nearest hotel: No hotel found
tionk essil - nearest hotel: Campement Edonia
bikaner - nearest hotel: Hotel Taj Restaurant
mulayjah - nearest hotel: No

,City,Country,Lat,Lng,Humidity,Hotel Name
34,trinidad,BO,-14.8333,-64.9000,79,Hotel Palace
42,port hedland,AU,-20.3167,118.5667,47,The Esplanade Hotel
63,san ignacio rio muerto,MX,27.4167,-110.2333,25,No hotel found
76,bandarbeyla,SO,9.4942,50.8122,76,No hotel found
100,tamanrasset,DZ,22.7850,5.5228,10,فندق الأمان
107,kone,NC,-21.0595,164.8658,57,Pacifik Appartels
142,portland,US,45.5234,-122.6762,43,the Hoxton
146,laguna,US,38.4210,-121.4238,35,Holiday Inn Express & Suites
157,hamilton,US,39.1834,-84.5333,79,North Vista Manor
158,marka,SO,1.7159,44.7717,75,Salsabiil Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
#https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot#:~:text=You%20can%20use%20keyword%20hover_cols%20to%20add%20additional%20columns%20to%20your%20hover.&text=Additional%20columns%20to%20add%20to,indexes%20if%20use_index%20is%20True).

# Configure the map plot
hotel_mappy=hotel_df.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=1,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                   xlim=(-180,180),ylim=(-60,60),hover_cols=['Country','Hotel Name'])


# Display the map
hotel_mappy

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)